In [1]:
from rdkit import Chem
from openff.toolkit import ForceField, Molecule, Topology
from Simulation.helper_functions import get_gnn_sim, MODEL_PATH, SOLVENT_DICT
from openmm.app import HBonds
from rdkit import Chem
from openff.toolkit import ForceField, Molecule, Topology
from Simulation.helper_functions import get_gnn_sim, MODEL_PATH, SOLVENT_DICT, create_vac_sim, create_gnn_sim
from openmm.app import HBonds

In [2]:
force_field = "openff_no_water-3.0.0-alpha0.offxml"

In [3]:
input_pdb = "3QTU/receptor.pdb"
output_pdb = "3QTU/receptor_off.pdb"

res_rename_map = {
    "BCB": "GLY",
    "BCC": "NME",
    "BNC": "ACE",
}

with open(input_pdb) as fin, open(output_pdb, "w") as fout:
    for line in fin:
        if not line.startswith(("ATOM")):   # note: this excludes HETATM lines
            fout.write(line)
            continue

        # Current residue and atom names from the PDB columns
        resname = line[17:20].strip()
        atomname = line[12:16].strip()

        # --- Rename residues: BCB -> GLY, BCC -> NME, BNC -> ACE ---
        if resname in res_rename_map:
            new_resname = res_rename_map[resname]
            # Replace residue name in its fixed-width field (cols 18–20)
            line = line[:17] + f"{new_resname:>3}" + line[20:]
            resname = new_resname  # update variable so later logic sees GLY/NME/ACE

        # --- Fix GLY: rename HA/HB -> HA2/HA3 ---
        if resname == "GLY":
            if atomname == "HA":
                new_atomname = "HA2"
                line = line[:12] + f"{new_atomname:>4}" + line[16:]
            elif atomname == "HB":
                new_atomname = "HA3"
                line = line[:12] + f"{new_atomname:>4}" + line[16:]

        # --- Fix ACE hydrogen names ---
        if resname == "ACE" and atomname in {"HA", "HB", "H2"}:
            new_atomname = "H" + str(["HA", "HB", "H2"].index(atomname) + 1)
            line = line[:12] + f"{new_atomname:>4}" + line[16:]

        # --- Fix NME hydrogen names ---
        if resname == "NME" and atomname in {"HA", "HB", "H2"}:
            new_atomname = "H" + str(["HA", "HB", "H2"].index(atomname) + 1)
            line = line[:12] + f"{new_atomname:>4}" + line[16:]

        fout.write(line)


In [4]:
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Load your current receptor
fixer = PDBFixer(filename="3QTU/receptor_optC.pdb")

# Identify missing stuff and make the protein “standard”
fixer.findMissingResidues()
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(pH=7.0)

# Write out a cleaned, fully standardised PDB
with open("3QTU/receptor_fixed.pdb", "w") as out:
    PDBFile.writeFile(fixer.topology, fixer.positions, out)

In [5]:
ligand = Molecule.from_file("3QTU/ligand.sdf", allow_undefined_stereo=True)

In [6]:
# Use the cleaned receptor
# top = Topology.from_pdb("3QTU/receptor_fixed.pdb", unique_molecules=[ligand])
protein_top = Topology.from_pdb("3QTU/receptor_fixed.pdb")

In [7]:
# complex_top = Topology.from_molecules([top.molecule(0), top.molecule(-1)])
complex_top = Topology.from_molecules(list(protein_top.molecules) + [ligand])

In [8]:
ligand_top = Topology.from_molecules([complex_top.molecule(-1)])
receptor_top = Topology.from_molecules(list(protein_top.molecules))

In [9]:
def make_sim(off_topology, solvent, num_confs=1):
    sim = get_gnn_sim(
        off_topology=off_topology,
        solvent=solvent,
        model_path=MODEL_PATH,
        solvent_dict=SOLVENT_DICT,
        cache=None,
        save_name=None,
        partial_charges=None,
        forcefield=force_field,
        constraints=HBonds,
        num_confs=num_confs,
    )
    return sim

In [10]:
def single_energy(sim):
    return sim._ref_system.calculate_energy()._value

In [11]:
# Solvent (e.g., TIP3P)
complex_solv = single_energy(make_sim(complex_top, "tip3p", num_confs=1))
ligand_solv = single_energy(make_sim(ligand_top, "tip3p", num_confs=1))
receptor_solv = single_energy(make_sim(receptor_top, "tip3p", num_confs=1))

# Vacuum
complex_vac = single_energy(make_sim(complex_top, "vac", num_confs=1))
ligand_vac = single_energy(make_sim(ligand_top, "vac", num_confs=1))
receptor_vac = single_energy(make_sim(receptor_top, "vac", num_confs=1))

/home/campus.ncl.ac.uk/c2033567/Documents/GNNImplicitSolvent/Simulation/helper_functions.py:1160: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(model

Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges based on AM1BCC


Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recognize residue GLU; did you forget to call .add_molecules() to add it?
Did not recognize residue LYS; did you forget to call .add_molecules() to add it?
Did not recognize residue ILE; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue GLU; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue THR; did you forget to call .add_molecules() to add it?
Did not recognize residue TYR; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recogniz

Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recognize residue THR; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue TYR; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue ILE; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue LEU; did you forget to call .add_molecules() to add it?
Did not recognize residue ARG; did you forget to call .add_molecules() to add it?
Did not recogniz

Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges based on AM1BCC


...

Did not recognize residue ILE; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue LEU; did you forget to call .add_molecules() to add it?
Did not recognize residue ARG; did you forget to call .add_molecules() to add it?
Did not recognize residue LEU; did you forget to call .add_molecules() to add it?


Platform:  GPU  ready
Using OpenFF forcefield: openff_no_water-3.0.0-alpha0.offxml
setting charges based on AM1BCC


Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recognize residue GLU; did you forget to call .add_molecules() to add it?
Did not recognize residue LYS; did you forget to call .add_molecules() to add it?
Did not recognize residue ILE; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue GLU; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue THR; did you forget to call .add_molecules() to add it?
Did not recognize residue TYR; did you forget to call .add_molecules() to add it?
Did not recognize residue GLY; did you forget to call .add_molecules() to add it?
Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recogniz

Did not recognize residue VAL; did you forget to call .add_molecules() to add it?
Did not recognize residue THR; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue TYR; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue ILE; did you forget to call .add_molecules() to add it?
Did not recognize residue NME; did you forget to call .add_molecules() to add it?
Did not recognize residue ACE; did you forget to call .add_molecules() to add it?
Did not recognize residue LEU; did you forget to call .add_molecules() to add it?
Did not recognize residue ARG; did you forget to call .add_molecules() to add it?
Did not recogniz

Platform:  GPU  ready
Platform:  GPU  ready
Platform:  GPU  ready
All parallel systems have the same forces as the reference System


In [12]:
# Solvation contribution to binding
ddG_solv = (complex_solv - complex_vac) - (ligand_solv - ligand_vac) - (receptor_solv - receptor_vac)

In [13]:
kj_to_kcal = 1 / 4.184

print(f"Complex in Solvent: {complex_solv * kj_to_kcal:.3f} kcal/mol")
print(f"Ligand in Solvent: {ligand_solv * kj_to_kcal:.3f} kcal/mol")
print(f"Receptor in Solvent: {receptor_solv * kj_to_kcal:.3f} kcal/mol")
print(f"Complex in Vacuum : {complex_vac * kj_to_kcal:.3f} kcal/mol")
print(f"Ligand in Vacuum : {ligand_vac * kj_to_kcal:.3f} kcal/mol")
print(f"Receptor in Vacuum : {receptor_vac * kj_to_kcal:.3f} kcal/mol")

print(f"Delta Delta G in Solvent = {ddG_solv * kj_to_kcal:.3f} kcal/mol")

Complex in Solvent: -853.550 kcal/mol
Ligand in Solvent: -434.620 kcal/mol
Receptor in Solvent: -417.988 kcal/mol
Complex in Vacuum : 432.095 kcal/mol
Ligand in Vacuum : -376.991 kcal/mol
Receptor in Vacuum : 886.290 kcal/mol
Delta Delta G in Solvent = 76.261 kcal/mol


In [14]:
from rdkit import Chem
from openff.toolkit import ForceField, Molecule, Topology
from Simulation.helper_functions import get_gnn_sim, MODEL_PATH, SOLVENT_DICT, create_vac_sim, create_gnn_sim
from openmm.app import HBonds
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# NEW: imports for charge extraction from OpenMM System
from openmm import unit
from openmm import openmm as mm

def get_charges_from_openff_topology(off_topology, ff):
    """
    off_topology: openff.toolkit.Topology
    ff: openff.toolkit.ForceField
    returns: list of floats, one per atom, in units of elementary charge
    """
    system = ff.create_openmm_system(off_topology)

    nonbonded = None
    for i in range(system.getNumForces()):
        force = system.getForce(i)
        if isinstance(force, mm.NonbondedForce):
            nonbonded = force
            break
    if nonbonded is None:
        raise RuntimeError("No NonbondedForce found in system – cannot extract charges.")

    charges = []
    for i in range(nonbonded.getNumParticles()):
        q, sigma, eps = nonbonded.getParticleParameters(i)
        charges.append(q.value_in_unit(unit.elementary_charge))

    return charges

# Instantiate the OpenFF ForceField
ff = ForceField(force_field)

# Get per-atom charges
receptor_charges = get_charges_from_openff_topology(receptor_top, ff)
ligand_charges = get_charges_from_openff_topology(ligand_top, ff)

# Print some info
print("\n=== Charges ===")
print(f"Number of receptor atoms: {len(receptor_charges)}")
print(f"Number of ligand atoms:   {len(ligand_charges)}")

print(f"Total receptor charge: {sum(receptor_charges): .4f} e")
print(f"Total ligand charge:   {sum(ligand_charges): .4f} e")


[17:35:26] WARNING: Proton(s) added/removed

[17:35:27] WARNING: Proton(s) added/removed

[17:35:29] WARNING: Proton(s) added/removed

[17:35:29] WARNING: Proton(s) added/removed

[17:35:30] WARNING: Proton(s) added/removed

[17:35:31] WARNING: Proton(s) added/removed

[17:35:32] WARNING: Proton(s) added/removed

[17:35:33] WARNING: Proton(s) added/removed




=== Charges ===
Number of receptor atoms: 1629
Number of ligand atoms:   44
Total receptor charge:  2.0000 e
Total ligand charge:   -0.0000 e
